In [1]:
# Liberias
import os
import pandas as pd
import datetime
import glob
import pandas_datareader.data as web
import numpy as np

In [160]:
# Función display
def displaydf(dataframe, cols = None, rows = 20):
    with pd.option_context("display.max_columns", cols):
        with pd.option_context("display.max_rows", rows):
            display(dataframe)
    return True

In [2]:
# Obtain the dates correspoding to each file by the name
def dates_for_files(path):
    abspath = os.path.abspath(path)
    file_names = [f[:-4] for f in os.listdir(abspath) if os.path.isfile(os.path.join(abspath, f))]
    dates = [i.strftime('%Y-%m-%d') for i in sorted([pd.to_datetime(i[8:]).date() for i in file_names])]
    return dates

In [3]:
# Read mutiple CSV files and creat one single DataFrame 
def multiple_csv(path):
    all_files = glob.glob(os.path.join(path, "*.csv"))
    file_list = []
    for file in all_files:
            df = pd.read_csv(file, usecols=[0, 3])
            df['Fecha'] =   file
            df['Fecha'] = [i.replace('files\\NAFTRAC_','') for i in df['Fecha']]
            df['Fecha'] = [i.replace('.csv','') for i in df['Fecha']]
            file_list.append(df)
    all_df = pd.concat(file_list, ignore_index=True)
    return  all_df

In [150]:
# Clean and replace tickers and info
def cleaning_data(df):
    df['Ticker'] = [i.replace('*','') for i in df['Ticker']]
    df['Ticker'] = df['Ticker'] + '.MX'
    df['Peso (%)'] = df['Peso (%)']/100
    
     # Tickers discrepancy
    df['Ticker'] = df['Ticker'].replace('LIVEPOLC.1.MX', 'LIVEPOLC-1.MX')
    df['Ticker'] = df['Ticker'].replace('MEXCHEM.MX', 'ORBIA.MX')
    df['Ticker'] = df['Ticker'].replace('SITESB.1.MX', 'SITESB-1.MX')
    df['Ticker'] = df['Ticker'].replace('GFREGIOO.MX', 'RA.MX')
    df['Ticker'] = df['Ticker'].replace('NMKA.MX', 'NEMAKA.MX')
    
    # Remove tickers for CASH
    tickers_drop = ['KOFL.MX', 'BSMXB.MX', 'MXN.MX', 'USD.MX','\xa0.MX','KOFUBL.MX']
    rows = list(df[list(df['Ticker'].isin(tickers_drop))].index)
    df.drop(rows, inplace=True)
    return df

In [5]:
# Adj. Closes from Yahoo Finance
def get_closes(tickers, start_date=None, end_date=None, freq=None):
    closes = pd.DataFrame(columns=tickers, index=web.YahooDailyReader(tickers[0], start=start_date, end=end_date
                                                                      , interval=freq).read().index)
    for ticker in tickers:
        df = web.YahooDailyReader(symbols=ticker, start=start_date, end=end_date, interval=freq).read()
        closes[ticker] = df['Adj Close']
    closes.index_name = 'Date'
    closes = closes.sort_index()
    return closes

In [6]:
# Tickers as index 
def global_tickers(data):
    tickers_list = list(data['Ticker'])
    return np.unique(tickers_list).tolist()

In [154]:
def fix_weight(historical):
    fix_weights = historical['Fecha'] == dates[0]
    fix_weights = historical[fix_weights]
    fix = fix_weights.drop(['Fecha', 'Precio'], axis=1)
    return  fix.rename(columns={'Peso (%)':'Peso (%) Fijo'})

In [155]:
# Global DataFrame with dates, prices, weights (fix and float) and tickers
def global_df(prices, dates, historical, fix):
    
    # Fix dates from historicals
    fix_dates = sorted(list(set(prices.index.astype(str).tolist()) & set(dates)))
    
    # Corresponding price to fix dates
    fix_prices = prices.iloc[[int(np.where(prices.index.astype(str) == i)[0]) for i in fix_dates]]
    fix_prices = fix_prices.reindex(sorted(fix_prices.columns), axis=1)
    
    # Match fix dates to prices
    match = 0
    historical['Precio'] = [fix_prices.iloc[match, fix_prices.columns.to_list().index(i)] for i in historical['Ticker']]

    # Merge fix weights column
    fix = fix_weight(historical)
    join = pd.merge(historical,  fix,  on ='Ticker',  how ='outer')
    join['Peso (%) Fijo'] =join['Peso (%) Fijo'].fillna(0)
    df = join.set_index('Fecha')
    df_final = df.sort_index(ascending=True)
    
    return df_final[['Ticker','Precio','Peso (%)','Peso (%) Fijo']]

In [152]:
# Info from files
path = r'files' 
dates = dates_for_files(path)
data_from_files = multiple_csv(path)
historical = cleaning_data(data_from_files)

In [23]:
# Info from Yahoo
tickers = global_tickers(historical)
start, end = '2018-01-01', '2021-01-24'
prices = get_closes(tickers, start, end, freq="d")

In [156]:
# Global data info from files + yahoo prices
data = global_df(prices, dates, historical,fix)

In [196]:
# Passive investment - fix weight
def passive_investment(k, c, data):
    data['Capital'] = round(data['Peso (%) Fijo'] * k - data['Peso (%) Fijo'] * k * c,2)
    data['Titulos'] = round(data['Capital'] / data['Precio'],0)
    data['Postura'] = round(data['Titulos'] * data['Precio'],2)
    data['Comision'] = round(data['Postura'] * c,2)
    data['Rend.'] = data['Capital'].pct_change().fillna(0)
    data['Rend. Acum'] = np.cumsum(data['Rend.']).fillna(0)
    return data.drop(['Peso (%)'], axis=1)

In [197]:
k , c = 1000000, 0.00125
passive_df = passive_investment(k, c, data)

In [198]:
passive_df

,Ticker,Precio,Peso (%) Fijo,Capital,Titulos,Postura,Comision,Rend.,Rend. Acum
Fecha,,,,,,,,,
2018-01-31,AMXL.MX,16.014746,0.1331,132933.62,8301.0,132938.40,166.17,0.000000,0.000000
2018-01-31,GRUMAB.MX,206.878021,0.0154,15380.75,74.0,15308.97,19.14,-0.884298,-0.884298
2018-01-31,AC.MX,123.355064,0.0175,17478.12,142.0,17516.42,21.90,0.136363,-0.747934
2018-01-31,VOLARA.MX,15.930000,0.0046,4594.25,288.0,4587.84,5.73,-0.737143,-1.485077
2018-01-31,GAPB.MX,169.949356,0.0251,25068.62,148.0,25152.50,31.44,4.456521,2.971444
...,...,...,...,...,...,...,...,...,...
2021-01-21,ALSEA.MX,60.197136,0.0083,8289.62,138.0,8307.20,10.38,-0.937641,inf
2021-01-21,ORBIA.MX,48.178917,0.0177,17677.88,367.0,17681.66,22.10,1.132532,inf
2021-01-21,MEGACPO.MX,79.104218,0.0084,8389.50,106.0,8385.05,10.48,-0.525424,inf
